
# Zero‑Shot, Few‑Shot, and CoT Prompting with Guardrails (Gemini API)

- Zero‑shot, few‑shot, and chain‑of‑thought (CoT) prompting with **Gemini Pro**.
- A **Marketing** scenario that outputs a JSON recommendation object.
- **Guardrails**: knowledge boundary, refusal rules, and JSON schema validation.
- **Self‑consistency** sampling and a minimal **evaluation** harness.

> Set your `GOOGLE_API_KEY` (from Google AI Studio) in `Secrets` tab.

In [ ]:
from google.colab import userdata
try:
    import google.generativeai as genai
    genai.configure(api_key=userdata.get('GOOGLE_API_KEY'))
    _GEMINI_READY = bool(userdata.get('GOOGLE_API_KEY'))
except Exception:
    print("Install google-generativeai to enable live calls.")
    _GEMINI_READY = False

In [ ]:
import json, textwrap, random
import pandas as pd
import numpy as np
from typing import Any, Dict, Tuple, List, Optional
from jsonschema import Draft202012Validator, ValidationError

MODEL_NAME = "gemini-2.5-flash"

def get_model(response_mime_type: Optional[str] = None, temperature: float = 0.3):
    if not _GEMINI_READY:
        return None
    cfg = {"temperature": temperature}
    if response_mime_type:
        cfg["response_mime_type"] = response_mime_type
    return genai.GenerativeModel(MODEL_NAME, generation_config=cfg)


## Sample KPI Data (Marketing Story)


In [ ]:
np.random.seed(1)
channels = ["Search","Social","Display","Affiliate","Email"]
kpi = pd.DataFrame({
    "channel": channels,
    "impressions": np.random.randint(1_000_000, 5_000_000, size=len(channels)),
    "clicks": np.random.randint(20_000, 120_000, size=len(channels)),
    "spend_usd": np.random.randint(10_000, 70_000, size=len(channels)),
    "conversions": np.random.randint(300, 2_500, size=len(channels)),
    "adstock_index": np.round(np.random.uniform(0.6, 1.3, size=len(channels)), 2),
})
kpi["ctr"] = (kpi["clicks"]/kpi["impressions"]).round(4)
kpi["cvr"] = (kpi["conversions"]/kpi["clicks"]).round(4).clip(0,1)
kpi["cpa"] = (kpi["spend_usd"]/kpi["conversions"]).round(2)
display(kpi)


## Guardrails: JSON Schema, Refusal Format, and Helpers


In [ ]:
SCHEMA = {
    "type":"object",
    "required":["brief","recommendation","red_flags"],
    "properties":{
        "brief":{"type":"string","maxLength":900},
        "recommendation":{
            "type":"object",
            "required":["proposed_shifts","confidence","risks","rationale_short"],
            "properties":{
                "proposed_shifts":{
                    "type":"array",
                    "items":{
                        "type":"object",
                        "required":["channel","delta_pct","reason"],
                        "properties":{
                            "channel":{"type":"string"},
                            "delta_pct":{"type":"number"},
                            "reason":{"type":"string"}
                        }
                    }
                },
                "confidence":{"type":"number","minimum":0,"maximum":1},
                "risks":{"type":"array","items":{"type":"string"}},
                "rationale_short":{"type":"array","items":{"type":"string"},"minItems":1,"maxItems":3}
            }
        },
        "red_flags":{
            "type":"array",
            "items":{
                "type":"object",
                "required":["signal","evidence"],
                "properties":{"signal":{"type":"string"},"evidence":{"type":"string"}}
            }
        }
    },
    "additionalProperties":False
}

REFUSAL_SCHEMA = {
    "type":"object",
    "required":["error","reason"],
    "properties":{
        "error":{"type":"string","enum":["insufficient_information"]},
        "reason":{"type":"string"}
    }
}

def is_valid_json(payload: Any, schema: Dict[str,Any]) -> Tuple[bool,str]:
    try:
        Draft202012Validator(schema).validate(payload)
        return True, ""
    except ValidationError as e:
        return False, f"{e.message} @ {list(e.path)}"

def kpi_to_xml(df: pd.DataFrame) -> str:
    rows = []
    for _,r in df.iterrows():
        row = "".join([f"<{c}>{r[c]}</{c}>" for c in df.columns])
        rows.append(f"<row>{row}</row>")
    return "<kpi>\n" + "\n".join(rows) + "\n</kpi>"

def build_notes_xml(text: str) -> str:
    return f"<notes>\n{text}\n</notes>"

def default_sales_notes() -> str:
    return ("Retail steady; enterprise slower. Affiliate bundle launched; "
            "email CTR strong but mobile checkout underperformed mid-week.")


## Zero‑Shot Prompt


In [ ]:
ZERO_SHOT_SYSTEM = textwrap.dedent("""
You are a marketing analytics assistant for weekly planning.
Use only the provided context. If required data are missing, return an insufficient_information object.
Emit valid JSON only - follow the given JSON schema exactly, no extra keys.
""").strip()

def zero_shot_prompt(kpi_xml: str, notes_xml: str) -> str:
    return textwrap.dedent(f"""
    ROLE: Marketing analytics assistant
    INSTRUCTIONS:
    - Summarize weekly performance in <=120 words (leadership-friendly).
    - Propose budget shifts per channel (delta_pct, positive=more), with short reasons.
    - List top risks.
    - Provide 1-3 bullets in 'rationale_short'.
    - Detect red flags: e.g., CTR up but CVR down, or abnormal CPA.

    KNOWLEDGE BOUNDARY: Use ONLY the context below.
    If insufficient, output the refusal object (error, reason).

    OUTPUT: Valid JSON only. No markdown. No extra commentary.
    SCHEMA (summary):
    {json.dumps(SCHEMA, indent=2)}

    CONTEXT START
    {kpi_xml}
    {notes_xml}
    CONTEXT END
    """).strip()

def get_model(response_mime_type: Optional[str] = None, temperature: float = 0.3):
    # redefined here for safety in some environments
    if not _GEMINI_READY:
        return None
    cfg = {"temperature": temperature}
    if response_mime_type:
        cfg["response_mime_type"] = response_mime_type
    return genai.GenerativeModel(MODEL_NAME, generation_config=cfg)

def run_zero_shot(df: pd.DataFrame, sales_notes: Optional[str] = None, temperature: float = 0.2):
    if not _GEMINI_READY:
        print("Gemini not configured - showing prompt head:")
        print(zero_shot_prompt(kpi_to_xml(df), build_notes_xml(sales_notes or default_sales_notes()))[:1000])
        return None
    model = get_model(response_mime_type="application/json", temperature=temperature)
    prompt = zero_shot_prompt(kpi_to_xml(df), build_notes_xml(sales_notes or default_sales_notes()))
    resp = model.generate_content([ZERO_SHOT_SYSTEM, prompt])
    try:
        data = json.loads(resp.text)
        ok, msg = is_valid_json(data, SCHEMA)
        print("JSON validity:", ok, msg)
        return data
    except Exception as e:
        print("Parse error:", e, "\nRaw:", getattr(resp, "text", str(resp)))
        return None

zero_shot_output = run_zero_shot(kpi)
# display(zero_shot_output)
print(json.dumps(zero_shot_output, indent=4))


## Few‑Shot Prompt (with 2 Demos, incl. Red‑Flag Case)


In [ ]:
DEMO_1_INPUT = {
  "kpi":[
    {"channel":"Search","ctr":0.028,"cvr":0.032,"cpa":65,"adstock_index":1.05},
    {"channel":"Social","ctr":0.035,"cvr":0.018,"cpa":130,"adstock_index":0.90}
  ],
  "notes":"Mobile checkout bug fixed mid-week."
}
DEMO_1_OUTPUT = {
  "brief":"Search efficient; Social high clicks but weak conversion.",
  "recommendation":{
    "proposed_shifts":[
      {"channel":"Search","delta_pct":10,"reason":"Efficient CPA and steady CVR"},
      {"channel":"Social","delta_pct":-10,"reason":"High CTR but low CVR; stabilize funnel first"}
    ],
    "confidence":0.72,
    "risks":["Attribution noise around fix"],
    "rationale_short":["Favor efficient lanes","Mitigate funnel weakness"]
  },
  "red_flags":[{"signal":"High CTR but low CVR","evidence":"Social CTR 0.035 vs CVR 0.018"}]
}

DEMO_2_INPUT = {
  "kpi":[
    {"channel":"Display","ctr":0.012,"cvr":0.026,"cpa":95,"adstock_index":1.10},
    {"channel":"Email","ctr":0.055,"cvr":0.041,"cpa":40,"adstock_index":0.80}
  ],
  "notes":"Partner newsletter featuring bundles."
}
DEMO_2_OUTPUT = {
  "brief":"Email strong engagement and conversion; Display stable.",
  "recommendation":{
    "proposed_shifts":[
      {"channel":"Email","delta_pct":8,"reason":"Best CPA and engagement"},
      {"channel":"Display","delta_pct":-5,"reason":"Maintain but refresh creatives"}
    ],
    "confidence":0.78,
    "risks":["List fatigue risk"],
    "rationale_short":["Exploit performers","Conserve weaker lanes"]
  },
  "red_flags":[]
}

def few_shot_prompt(kpi_xml: str, notes_xml: str) -> str:
    demos = textwrap.dedent(f"""
    DEMO 1 INPUT:
    <kpi_demo>{json.dumps(DEMO_1_INPUT["kpi"])}</kpi_demo>
    <notes_demo>{DEMO_1_INPUT["notes"]}</notes_demo>
    DEMO 1 OUTPUT:
    {json.dumps(DEMO_1_OUTPUT)}

    DEMO 2 INPUT:
    <kpi_demo>{json.dumps(DEMO_2_INPUT["kpi"])}</kpi_demo>
    <notes_demo>{DEMO_2_INPUT["notes"]}</notes_demo>
    DEMO 2 OUTPUT:
    {json.dumps(DEMO_2_OUTPUT)}
    """).strip()

    return textwrap.dedent(f"""
    ROLE: Marketing analytics assistant
    INSTRUCTIONS: Follow schema and tone. Use only provided context. If insufficient, return refusal.
    OUTPUT: Valid JSON only.

    SCHEMA:
    {json.dumps(SCHEMA, indent=2)}

    {demos}

    LIVE INPUT:
    {kpi_xml}
    {notes_xml}
    """).strip()

def run_few_shot(df: pd.DataFrame, sales_notes: Optional[str] = None, temperature: float = 0.2):
    if not _GEMINI_READY:
        print("Gemini not configured - showing prompt head:")
        print(few_shot_prompt(kpi_to_xml(df), build_notes_xml(sales_notes or default_sales_notes()))[:1000])
        return None
    model = get_model(response_mime_type="application/json", temperature=temperature)
    prompt = few_shot_prompt(kpi_to_xml(df), build_notes_xml(sales_notes or default_sales_notes()))
    resp = model.generate_content(prompt)
    try:
        data = json.loads(resp.text)
        ok, msg = is_valid_json(data, SCHEMA)
        print("JSON validity:", ok, msg)
        return data
    except Exception as e:
        print("Parse error:", e, "\nRaw:", getattr(resp, "text", str(resp)))
        return None

few_shot_output = run_few_shot(kpi)
print(json.dumps(few_shot_output, indent=4))


## Chain‑of‑Thought (Internal) + Brief Rationale


In [ ]:
COT_INTERNAL = """
Reason step-by-step internally. Do NOT reveal your chain of thought.
Output only the final JSON matching the schema, with 1-3 bullets in 'rationale_short'.
"""

def cot_prompt(kpi_xml: str, notes_xml: str) -> str:
    return textwrap.dedent(f"""
    ROLE: Senior marketing analyst
    INSTRUCTIONS:
    - {COT_INTERNAL.strip()}
    - Sanity-check conflicting signals (e.g., CTR up but CVR down) before deciding.

    OUTPUT: Valid JSON only. No extra commentary.
    SCHEMA: {json.dumps(SCHEMA, indent=2)}
    CONTEXT:
    {kpi_xml}
    {notes_xml}
    """).strip()

def run_cot(df: pd.DataFrame, sales_notes: Optional[str] = None, temperature: float = 0.3):
    if not _GEMINI_READY:
        print("Gemini not configured - showing prompt head:")
        print(cot_prompt(kpi_to_xml(df), build_notes_xml(sales_notes or default_sales_notes()))[:1000])
        return None
    model = get_model(response_mime_type="application/json", temperature=temperature)
    prompt = cot_prompt(kpi_to_xml(df), build_notes_xml(sales_notes or default_sales_notes()))
    resp = model.generate_content(prompt)
    try:
        data = json.loads(resp.text)
        ok, msg = is_valid_json(data, SCHEMA)
        print("JSON validity:", ok, msg)
        return data
    except Exception as e:
        print("Parse error:", e, "\nRaw:", getattr(resp, "text", str(resp)))
        return None

cot_output = run_cot(kpi)
print(json.dumps(cot_output, indent=4))


## Self‑Consistency Sampling


In [ ]:
def extract_vote_key(obj: Dict[str,Any]) -> str:
    try:
        shifts = obj["recommendation"]["proposed_shifts"]
        top = sorted(shifts, key=lambda s: s.get("delta_pct",0), reverse=True)[0]
        return top["channel"] if top["delta_pct"] > 0 else "none"
    except Exception:
        return "none"

def run_self_consistency(df: pd.DataFrame, sales_notes: Optional[str] = None, M: int = 5, temperature: float = 0.7):
    if not _GEMINI_READY:
        print("Gemini not configured - skipping live sampling.")
        return None, None
    model = get_model(response_mime_type="application/json", temperature=temperature)
    prompt = cot_prompt(kpi_to_xml(df), build_notes_xml(sales_notes or default_sales_notes()))
    votes, results = [], []
    for _ in range(M):
        resp = model.generate_content(prompt)
        try:
            data = json.loads(resp.text)
            ok, _ = is_valid_json(data, SCHEMA)
            if ok:
                results.append(data)
                votes.append(extract_vote_key(data))
        except Exception:
            pass
    counts = {}
    for v in votes:
        counts[v] = counts.get(v,0)+1
    winner = max(counts, key=counts.get) if counts else "none"
    print("Votes:", counts, "-> winner:", winner)
    return results, {"winner": winner, "counts": counts}

sc_results, sc_summary = run_self_consistency(kpi, M=5, temperature=0.7)
display(sc_summary)